In [1]:
%load_ext autoreload
%autoreload 2
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from dotenv import dotenv_values
import sys
sys.path.append('src/')
!pip install --upgrade reacher

In [5]:
from util import get_ml_client
ml_client = get_ml_client()

In [2]:
from reacher.reacher import RemoteClient, Reacher
from dotenv import dotenv_values

config = dotenv_values()  # take environment variables from .env.

reacher = Reacher(
    build_name="pytorch_base",
    image_name="pytorch_base",
    build_context="dockercontext",
    host=config["HOST"],
    user=config["USER"],
    password=config["PASSWORD"],
    ssh_key_filepath=config["SSH_KEY_PATH"],
)

In [36]:
reacher.build()


[WARNING]: Empty continuation line found in:
    RUN apt-get update && apt-get -y install     curl RUN curl -sL https://aka.ms/InstallAzureCLIDeb | bash
[WARNING]: Empty continuation lines will become errors in a future release.
Step 1/9 : FROM pytorch/pytorch as base
 ---> 71eb2d092138
Step 2/9 : WORKDIR /workspace
 ---> Using cache
 ---> b4c2d4774a4c
Step 3/9 : ENV DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 1f34b394c77c
Step 4/9 : RUN apt-get update && apt-get -y install     curl RUN curl -sL https://aka.ms/InstallAzureCLIDeb | bash
 ---> Using cache
 ---> 84ad48dd2000
Step 5/9 : RUN pip3 install pandas
 ---> Using cache
 ---> 536cf169e1e0
Step 6/9 : FROM base
 ---> 536cf169e1e0
Step 7/9 : COPY requirements.txt /tmp
 ---> Using cache
 ---> af8b4d334673
Step 8/9 : RUN pip install -r /tmp/requirements.txt
 ---> Using cache
 ---> 63c29a2660a6
Step 9/9 : RUN apt-get -y install screen
 ---> Using cache
 ---> 8981f8b335cd
Successfully built 8981f8b335cd
Successfully tagged pyt

In [3]:
reacher.setup(
    ports=[8888, 6666],
    envs=dotenv_values(".env") 
)

docker: Error response from daemon: Conflict. The container name "/pytorch_base" is already in use by container "453125a248c17c4882d313908cfe7fa5083eceda713a4945d1321719c1bbe8e6". You have to remove (or rename) that container to be able to reuse that name.

See 'docker run --help'.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
screen is already the newest version (4.6.2-1ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
reacher.execute(
    context_folder="src",
    file="src/setup.py",
    command="python setup.py --dataset_name twitter-sentiment",
    named_session="setup_twitter_dataset",
)

In [ ]:
reacher.execute(
    context_folder="src",
    file="src/train.py",
    command="python package.py --dataset twitter-sentiment --lock_embedding --lock_first_n_layers 7 --batch_size 25 --iterations 50000",
    named_session="train_sentiment_model",
)

In [ ]:
reacher.execute(
    file="dockercontext_torchserve/requirements.txt",
    context_folder="src",
    command="python package.py --model_name xlmr_sentiment_traced --version 1 --handler handler.py --requirements requirements.txt",
    named_session="package_sentiment_model",
)

=(BYour file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'https://clusterwstorage396d8d4e8.blob.core.windows.net/azureml-blobstore-88938f56-fe02-4fe5-8ee5-3cc95860c6b4/LocalUpload/107b67ecae88ae11cb90a61742fc7cc5/traced.pt' 'artifacts' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


In [4]:
reacher.artifacts

['testTwitter', 'twitter-sentiment']

In [196]:
reacher.get_artifact("sentiment.mar", "artifacts")

In [ ]:
reacher.execute_command(
    "jupyter notebook --ip 0.0.0.0 --allow-root --port 8888",
    named_session="jupyter"
)